In [1]:
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
year = '2016'

In [19]:
df = pd.read_csv('data/points_against_'+year+'.csv')

In [20]:
step = 1
df.loc[((df.Position == 'WR') | (df.Position == 'RB')) & (df.Points_Against>46), 'Points_Against'] = 44;
df.loc[((df.Position == 'WR') | (df.Position == 'RB')) & (df.Points_Against<0), 'Points_Against'] = -1;
df.loc[((df.Position == 'QB') & (df.Points_Against>36)), 'Points_Against'] = 36;
df.loc[((df.Position == 'QB') & (df.Points_Against<0)), 'Points_Against'] = -1;
df.loc[((df.Position == 'TE') | (df.Position == 'K') | (df.Position == 'DEF')) & (df.Points_Against>26), 'Points_Against'] = 24;
df.loc[((df.Position == 'TE') | (df.Position == 'K') | (df.Position == 'DEF')) & (df.Points_Against<0), 'Points_Against'] = -1;
df.Points_Against = (df.Points_Against/step).astype(int)

In [21]:
# Check what is the best indicator
for pos in df.Position.unique():
    df_pos = df.loc[(df.Position == pos) & (df.Game_Count==1)]
    pos_mean = df_pos.Points_Against.astype(float).mean()
    for team in df.Team_Against.unique():
        pa_mean = df_pos.loc[(df_pos.Team_Against == team),'Points_Against'].astype(float).mean()
        df_pa = pd.DataFrame({'Position': [pos], 'Points_Against': [pa_mean], 'Team_Against':team, 'VS':'Ave', 'Week':-1});
    
        vs_mean = df_pos.loc[(df_pos.VS == team),'Points_Against'].astype(float).mean()
        df_vs = pd.DataFrame({'Position': [pos], 'Points_Against': [vs_mean], 'Team_Against':'Ave', 'VS':team, 'Week':-1});
        
        df = pd.concat([df, df_pa, df_vs])
    df = pd.concat([df, pd.DataFrame({'Position': [pos], 'Points_Against': [pos_mean], 'Team_Against':'Ave', 'VS':'Ave', 'Week':-1})])

In [22]:
def matchup(team_1, team_2):
    matchup = pd.DataFrame();
    for pos in df.Position.unique():
        team_1_pa = df.loc[(df.Team_Against == team_1) & (df.VS == 'Ave') & (df.Position == pos), 'Points_Against'].astype(float).mean();
        team_2_pa = df.loc[(df.Team_Against == team_2) & (df.VS == 'Ave') & (df.Position == pos), 'Points_Against'].astype(float).mean();
        team_1_vs = df.loc[(df.Team_Against == 'Ave') & (df.VS == team_1) & (df.Position == pos), 'Points_Against'].astype(float).mean();
        team_2_vs = df.loc[(df.Team_Against == 'Ave') & (df.VS == team_2) & (df.Position == pos), 'Points_Against'].astype(float).mean();
        pos_mean = df.loc[(df.Team_Against == 'Ave') & (df.VS == 'Ave') & (df.Position == pos), 'Points_Against'].astype(float).mean();
        df_pos = pd.DataFrame([{'Position':pos, 'Team_Against': team_1, 'VS': team_2, 'Points_Against': (team_1_pa*team_2_vs/pos_mean)},{'Position':pos, 'Team_Against': team_2, 'VS': team_1, 'Points_Against': (team_2_pa*team_1_vs/pos_mean)}])
        
        matchup = pd.concat([matchup,df_pos])
    
    matchup['Range_Low'] = (matchup['Points_Against'].astype(int))*step
    matchup['Range_High'] = (matchup['Points_Against'].astype(int)+1)*step
    return matchup[['Team_Against','VS','Position','Range_Low','Range_High']].reset_index(drop=True)
        

In [23]:
print(matchup('GB','MIN'));
#df[(df.Game_Count==1)&(df.Team_Against=='GB')&(df.Position=='QB')]

   Team_Against   VS Position  Range_Low  Range_High
0            GB  MIN       QB         15          16
1           MIN   GB       QB         20          21
2            GB  MIN       RB         11          12
3           MIN   GB       RB         12          13
4            GB  MIN       WR         26          27
5           MIN   GB       WR         26          27
6            GB  MIN       TE          9          10
7           MIN   GB       TE          4           5
8            GB  MIN        K          7           8
9           MIN   GB        K          7           8
10           GB  MIN      DEF          6           7
11          MIN   GB      DEF          6           7


In [24]:
schedule_year = 2017
df_schedule = pd.read_csv('data/schedule_'+str(schedule_year)+'.csv')


In [27]:
df_tst = pd.read_csv('data/points_against_'+str(schedule_year)+'.csv')
df_tst['Pred_l'] = 0;
df_tst['Pred_h'] = 0;
df_index = df_tst.loc[df_tst.Game_Count == 1][['Team_Against','VS','Week']].copy()
df_index = df_index.drop_duplicates().reset_index(drop=True)
for i in range(len(df_index)):
    team_1 = df_index.iloc[i].Team_Against
    team_2 = df_index.iloc[i].VS
    wk = df_index.iloc[i].Week
    df_pred = matchup(team_1,team_2)
    for j in range(len(df_pred)):
        ta = df_pred.iloc[j].Team_Against
        vs = df_pred.iloc[j].VS
        pos = df_pred.iloc[j].Position
        df_tst.loc[((df_tst.Team_Against == ta) & (df_tst.VS == vs) & (df_tst.Week == wk)), 'Pred_l'] = df_pred.iloc[j].Range_Low
        df_tst.loc[((df_tst.Team_Against == ta) & (df_tst.VS == vs) & (df_tst.Week == wk)), 'Pred_h'] = df_pred.iloc[j].Range_High

In [28]:
pos_list = ['QB','WR','RB','TE','K','DEF']
for pos in pos_list:
    print('Position: ' + pos + ' with step size of ' + str(step))
    df_eval = df_tst.loc[df_tst.Position==pos].copy()
    df_eval['Dev0'] = 0
    df_eval.loc[(df_eval.Points_Against>df_eval.Pred_l) & (df_eval.Points_Against<df_eval.Pred_h), 'Dev0'] = 1
    df_eval['Dev1'] = 0
    df_eval.loc[(df_eval.Points_Against>df_eval.Pred_l-step) & (df_eval.Points_Against<df_eval.Pred_h+step), 'Dev1'] = 1
    df_eval['Dev2'] = 0
    df_eval.loc[(df_eval.Points_Against>df_eval.Pred_l-2*step) & (df_eval.Points_Against<df_eval.Pred_h+2*step), 'Dev2'] = 1
    df_eval['Dev5'] = 0
    df_eval.loc[(df_eval.Points_Against>df_eval.Pred_l-5*step) & (df_eval.Points_Against<df_eval.Pred_h+5*step), 'Dev5'] = 1
    print('Dev 0:'+str(df_eval.Dev0.mean()))
    print('Dev 1:'+str(df_eval.Dev1.mean()))
    print('Dev 2:'+str(df_eval.Dev2.mean()))
    print('Dev 5:'+str(df_eval.Dev5.mean()))

Position: QB with step size of 1
Dev 0:0.03125
Dev 1:0.109375
Dev 2:0.1796875
Dev 5:0.375
Position: WR with step size of 1
Dev 0:0.0078125
Dev 1:0.0390625
Dev 2:0.1015625
Dev 5:0.1953125
Position: RB with step size of 1
Dev 0:0.0234375
Dev 1:0.09375
Dev 2:0.1328125
Dev 5:0.2734375
Position: TE with step size of 1
Dev 0:0.046875
Dev 1:0.1875
Dev 2:0.3046875
Dev 5:0.6171875
Position: K with step size of 1
Dev 0:0.0
Dev 1:0.140625
Dev 2:0.3359375
Dev 5:0.6484375
Position: DEF with step size of 1
Dev 0:0.0
Dev 1:0.1484375
Dev 2:0.3125
Dev 5:0.625


In [32]:
wk = 5
def get_week(wk):
    df_tmp = df_schedule[df_schedule['Week'] == wk].copy()

    df_tmp['Team_1'] = df_tmp['Away']
    df_tmp['Team_2'] = df_tmp['Home']
    
    return df_tmp[['Team_1','Team_2']].reset_index(drop=True)

In [33]:
df_week = get_week(wk)
df_wk_matchups = pd.DataFrame()
for i in range(len(df_week)):
    df_tmp = matchup(df_week.Team_1[i], df_week.Team_2[i])
    df_wk_matchups = pd.concat([df_wk_matchups,df_tmp])

In [38]:
df_wk_matchups[df_wk_matchups.Position=='WR'].sort_values('Range_Low',ascending=False)

,Team_Against,VS,Position,Range_Low,Range_High
4,DAL,GB,WR,31,32
4,PHI,ARI,WR,30,31
5,GB,DAL,WR,29,30
5,TEN,MIA,WR,26,27
5,BAL,OAK,WR,26,27
4,LA,SEA,WR,25,26
4,CLE,NYJ,WR,24,25
5,CAR,DET,WR,24,25
5,SF,IND,WR,24,25
5,LAC,NYG,WR,23,24
